In [1]:
import pandas as pd
import numpy as np
import hyperopt
import os
import sys
import csv

from time import time

from hyperopt import hp, fmin, tpe
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import cross_val_score

In [2]:
df = pd.read_csv('data/manipulated_train.csv')

In [3]:
X = df.drop(['SalePrice'], axis=1)
y = df['SalePrice']

In [4]:
X.shape

(1460, 79)

In [5]:
np.sqrt(79)

8.8881944173155887

In [6]:
max_evals = 20
run_counter = 0
num_cv = 3
output_file = '/tmp/hyperopt-et.csv'

In [7]:
def run_wrapper(params):
    global run_counter
    global o_f
    
    run_counter += 1
    print "=============================="
    print "run", run_counter
    
    s = time()
    r2score = run_test(params)
    
    print
    print "R2 score:", r2score
    print "elapsed: {}s \n".format(int(round(time() - s)))
    
    writer.writerow([r2score] + list(params))
    o_f.flush()
    return r2score

In [8]:
def run_test( params ):
    max_depth, max_features, min_samples_leaf = params
    max_depth = int(max_depth)
    min_samples_leaf = int(min_samples_leaf)
    
    print "max_depth:", max_depth
    print "max_feature:", max_features
    print "min_samples_leaf:", min_samples_leaf
    
    et = ExtraTreesRegressor(max_depth=max_depth, max_features=max_features,
                             min_samples_leaf=min_samples_leaf, n_estimators=100)
    cvs = cross_val_score(et, X, y, cv=num_cv, n_jobs=-1)
    
    return 1 - np.mean(cvs)

In [ ]:
space = (
    hp.uniform('max_depth', 5, 30),
    hp.uniform('max_features', 0.1, 1),
    hp.uniform('min_samples_leaf', 1, 10)
)

In [ ]:
if __name__ == '__main__':
    
    headers = ['r2score', 'max_depth']
    o_f = open(output_file, 'wb')
    writer = csv.writer(o_f)
    writer.writerow(headers)
    
    start_time = time()
    best = fmin(run_wrapper, space, algo=tpe.suggest, max_evals=max_evals)
    end_time = time()
    
    print "Seconds passed:", int( round(end_time - start_time))
    #print "Best run:", optimizer.get_best_run()
    print best
    #print run_test( hyperopt.space_eval( space, best ))

run 1
max_depth: 23
max_feature: 0.768143084591
min_samples_leaf: 6

R2 score: 0.138550897162
elapsed: 1s 

run 2
max_depth: 16
max_feature: 0.504476991729
min_samples_leaf: 4

R2 score: 0.133722306708
elapsed: 1s 

run 3
max_depth: 16
max_feature: 0.258679882947
min_samples_leaf: 9

R2 score: 0.162824354189
elapsed: 1s 

run 4
max_depth: 12
max_feature: 0.705214931083
min_samples_leaf: 7

R2 score: 0.142111667649
elapsed: 1s 

run 5
max_depth: 13
max_feature: 0.781886245967
min_samples_leaf: 1

R2 score: 0.12472930484
elapsed: 2s 

run 6
max_depth: 29
max_feature: 0.737416939395
min_samples_leaf: 1

R2 score: 0.126903380548
elapsed: 2s 

run 7
max_depth: 5
max_feature: 0.299915340817
min_samples_leaf: 3

R2 score: 0.189220801951
elapsed: 1s 

run 8
max_depth: 20
max_feature: 0.144749307259
min_samples_leaf: 7
